In [1]:
import numpy as np
import pandas as pd
import re
import os

import os
import datetime
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(u'ggplot')
# import seaborn as sns
# import eli5
# import shap

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import roc_auc_score

import xgboost
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [3]:
pd.options.display.max_columns = 999

In [4]:
df=pd.read_csv('/Users/samuelaxa/Downloads/flight_delays_data.csv')
airport = pd.read_csv('/Users/samuelaxa/Downloads/airports.csv')
airline = pd.read_csv('/Users/samuelaxa/Downloads/airlines.csv',header=None, names=['index','tmp1','tmp2','tmp3','tmp4','tmp5','tmp6','tmp7'])

In [5]:
df['Airline_update']=np.where(df.Airline.notnull()==True, df.Airline, df.flight_no.str[0:2])

In [6]:
airport2 = airport[airport.type != 'closed'][airport.iata_code.notnull()][airport.iata_code != '0']
airport2.sort_values(by=['iata_code','latitude_deg'],ascending=False)
airport3=airport2.drop_duplicates('iata_code')

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [7]:
airline2=airline.drop_duplicates('tmp3')

In [8]:
df_copy= df.copy()

In [9]:
# df=df_copy

In [10]:
df = df.merge(airport3,left_on = 'Arrival', right_on = 'iata_code',how='left').merge(airline2,left_on='Airline_update',right_on='tmp3', how='left')


In [11]:
print df.shape
print df_copy.shape

(899114, 37)
(899114, 11)


In [12]:
df.head(2)

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,Airline_update,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,index,tmp1,tmp2,tmp3,tmp4,tmp5,tmp6,tmp7
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,UO,5536,RJBB,large_airport,Kansai International Airport,34.427299,135.244003,26.0,AS,JP,JP-27,Osaka,yes,RJBB,KIX,NaN,http://www.kansai-airport.or.jp/en/index.asp,http://en.wikipedia.org/wiki/Kansai_Internatio...,OSA,2774.0,Hong Kong Express Airways,\N,UO,HKE,HONGKONG SHUTTLE,Hong Kong SAR of China,Y
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,CI,5522,RCNN,medium_airport,Tainan Airport,22.950399,120.206001,63.0,AS,TW,TW-TNN,Tainan City,yes,RCNN,TNN,NaN,http://www.tna.gov.tw/,http://en.wikipedia.org/wiki/Tainan_Airport,"臺南機場, 臺南航空站",1756.0,China Airlines,\N,CI,CAL,DYNASTY,Taiwan,Y


In [13]:
#since it is more a binary targeting, so create new target as 1 and 0
df['Target'] = (df['is_claim'] == 800).astype(int)
#change flight_date to date_formate and extract month and date.
df['flight_date']=pd.to_datetime(df['flight_date'], format="%Y/%m/%d")
#delay_time to nummerical, cancel set all to 3
# df['feat_delay_time']=np.where(df['delay_time']=='Cancelled', 30, df['delay_time']).astype(float)
# delay_time is not something we know in advance and cannot use for prediction

In [14]:
df['feat_Week']=df['Week']
df['feat_std_hour']=df['std_hour']
df['feat_month']=df['flight_date'].dt.month
df['feat_day']=df['flight_date'].dt.day
df['feat_flight_no_len']=df.flight_no.str[2:].str.len()
df['cat_feat_day_of_week'] = df['flight_date'].dt.weekday_name

In [15]:
df.head()

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,Airline_update,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,index,tmp1,tmp2,tmp3,tmp4,tmp5,tmp6,tmp7,Target,feat_Week,feat_std_hour,feat_month,feat_day,feat_flight_no_len,cat_feat_day_of_week
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,UO,5536,RJBB,large_airport,Kansai International Airport,34.427299,135.244003,26.0,AS,JP,JP-27,Osaka,yes,RJBB,KIX,NaN,http://www.kansai-airport.or.jp/en/index.asp,http://en.wikipedia.org/wiki/Kansai_Internatio...,OSA,2774.0,Hong Kong Express Airways,\N,UO,HKE,HONGKONG SHUTTLE,Hong Kong SAR of China,Y,0,27,10,7,1,3,Friday
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,CI,5522,RCNN,medium_airport,Tainan Airport,22.950399,120.206001,63.0,AS,TW,TW-TNN,Tainan City,yes,RCNN,TNN,NaN,http://www.tna.gov.tw/,http://en.wikipedia.org/wiki/Tainan_Airport,"臺南機場, 臺南航空站",1756.0,China Airlines,\N,CI,CAL,DYNASTY,Taiwan,Y,0,17,11,4,23,4,Thursday
2,1582504,PR301,14,HKG,MNL,PR,11,0.0,2014-04-08,0,PR,5689,RPLL,large_airport,Ninoy Aquino International Airport,14.508600,121.019997,75.0,AS,PH,PH-U-A,Manila,yes,RPLL,MNL,NaN,NaN,http://en.wikipedia.org/wiki/Ninoy_Aquino_Inte...,Manila International Airport,3952.0,Philippine Airlines,\N,PR,PAL,PHILIPPINE,Philippines,Y,0,14,11,4,8,3,Tuesday
3,1582508,LD327,37,HKG,SIN,LD,3,0.1,2013-09-15,0,LD,26887,WSSS,large_airport,Singapore Changi Airport,1.350190,103.994003,22.0,AS,SG,SG-04,Singapore,yes,WSSS,SIN,NaN,http://www.changiairport.com/,https://en.wikipedia.org/wiki/Singapore_Changi...,NaN,191.0,Air Hong Kong,\N,LD,AHK,AIR HONG KONG,Hong Kong,N,0,37,3,9,15,3,Sunday
4,1582509,KA5390,40,HKG,PEK,KA,9,0.5,2015-10-05,0,KA,27188,ZBAA,large_airport,Beijing Capital International Airport,40.080101,116.584999,116.0,AS,CN,CN-11,Beijing,yes,ZBAA,PEK,NaN,http://en.bcia.com.cn/,http://en.wikipedia.org/wiki/Beijing_Capital_I...,"BJS, Bejing, Peking, Olympics",2056.0,Dragonair,\N,KA,HDA,Hong Kong Dragon Airlines,DRAGON,Y,0,40,9,10,5,4,Monday


In [16]:
#feature airline company feature
df['tmp6'].value_counts().head(10)

Hong Kong SAR of China    293623
DRAGON                    109852
China                      86163
United States              49788
Japan                      42756
Taiwan                     34998
Thailand                   25206
United Kingdom             21576
Finland                    20548
Malaysia                   19916
Name: tmp6, dtype: int64

In [17]:
#5% of the data
899114*0.05

44955.700000000004

In [18]:
df['cat_feat_flight_country']=np.where(df['tmp6'].isin(['Hong Kong SAR of China','DRAGON','China','United States','Japan']),df['tmp6'],'Others')

In [19]:
df['cat_feat_flight_country'].value_counts()

Others                    316932
Hong Kong SAR of China    293623
DRAGON                    109852
China                      86163
United States              49788
Japan                      42756
Name: cat_feat_flight_country, dtype: int64

In [20]:
#Airport vairable transformation

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899114 entries, 0 to 899113
Data columns (total 45 columns):
flight_id                  899114 non-null int64
flight_no                  899114 non-null object
Week                       899114 non-null int64
Departure                  899114 non-null object
Arrival                    899114 non-null object
Airline                    897400 non-null object
std_hour                   899114 non-null int64
delay_time                 899114 non-null object
flight_date                899114 non-null datetime64[ns]
is_claim                   899114 non-null int64
Airline_update             899114 non-null object
id                         899114 non-null int64
ident                      899114 non-null object
type                       899114 non-null object
name                       899114 non-null object
latitude_deg               899114 non-null float64
longitude_deg              899114 non-null float64
elevation_ft               886426 

In [22]:
airport.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [23]:
df.type.value_counts(dropna=False)

large_airport     842455
medium_airport     51282
small_airport       5377
Name: type, dtype: int64

In [24]:
df['cat_feat_type']=df['type']
df['feat_type_num']=np.where(df['type'] == 'large_airport', 3, np.where(df['type']=='medium_airport',2,1))
df['feat_internationl_airport']=df['name'].str.contains('International').astype(int)
df['feat_latitude_deg']=df['latitude_deg']
df['feat_longitude_deg']=df['longitude_deg']
df['cat_feat_iso_country']=np.where(df['iso_country'].isin(['CN','TW','JP','TH','AU','SG']),df['iso_country'],'Others')

In [25]:
df.iso_country.value_counts(dropna=False).head(10)

CN    229327
TW    105270
JP     84548
TH     74339
AU     48411
SG     44720
MY     40600
KR     39081
US     37282
PH     28845
Name: iso_country, dtype: int64

In [26]:
df.name.value_counts(dropna=False).head(5)

Taiwan Taoyuan International Airport     75751
Shanghai Pudong International Airport    61598
Suvarnabhumi Airport                     49730
Beijing Capital International Airport    45161
Singapore Changi Airport                 44720
Name: name, dtype: int64

In [27]:
# print df[df.latitude_deg.isnull()]
# print df[df.longitude_deg.isnull()]
# print df[df.elevation_ft.isnull()].head(5) 
#elevation_ft not able to use since have some missing value, need to have more time to check the meaning before filling value

In [28]:
df[df.feat_internationl_airport == 1].cat_feat_type.value_counts()

large_airport     701967
medium_airport     28340
Name: cat_feat_type, dtype: int64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899114 entries, 0 to 899113
Data columns (total 51 columns):
flight_id                    899114 non-null int64
flight_no                    899114 non-null object
Week                         899114 non-null int64
Departure                    899114 non-null object
Arrival                      899114 non-null object
Airline                      897400 non-null object
std_hour                     899114 non-null int64
delay_time                   899114 non-null object
flight_date                  899114 non-null datetime64[ns]
is_claim                     899114 non-null int64
Airline_update               899114 non-null object
id                           899114 non-null int64
ident                        899114 non-null object
type                         899114 non-null object
name                         899114 non-null object
latitude_deg                 899114 non-null float64
longitude_deg                899114 non-null float64


In [30]:
#change Airport information

In [31]:
#creating mean, lowerest, highest and take a mas

In [32]:
#dummy cat variable

In [33]:
feature= [y for y in df.columns if y[0:5]=='feat_']

In [34]:
for feat in feature:
    df['num_'+feat]=df[feat]

In [35]:
def dummified(df, col):
    df_tmp = pd.get_dummies(df[[col]])
    df_tmp.columns = ["num_feat_"+y for y in df_tmp.columns]
    return pd.concat([df, df_tmp], axis = 1)#df.join(df_tmp)

In [36]:
cat_feature= [y for y in df.columns if y[0:9]=='cat_feat_']

In [37]:
for cat_feat in cat_feature:
    df=dummified(df,cat_feat)

In [38]:
df.head(2)

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,Airline_update,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,index,tmp1,tmp2,tmp3,tmp4,tmp5,tmp6,tmp7,Target,feat_Week,feat_std_hour,feat_month,feat_day,feat_flight_no_len,cat_feat_day_of_week,cat_feat_flight_country,cat_feat_type,feat_type_num,feat_internationl_airport,feat_latitude_deg,feat_longitude_deg,cat_feat_iso_country,num_feat_Week,num_feat_std_hour,num_feat_month,num_feat_day,num_feat_flight_no_len,num_feat_type_num,num_feat_internationl_airport,num_feat_latitude_deg,num_feat_longitude_deg,num_feat_cat_feat_day_of_week_Friday,num_feat_cat_feat_day_of_week_Monday,num_feat_cat_feat_day_of_week_Saturday,num_feat_cat_feat_day_of_week_Sunday,num_feat_cat_feat_day_of_week_Thursday,num_feat_cat_feat_day_of_week_Tuesday,num_feat_cat_feat_day_of_week_Wednesday,num_feat_cat_feat_flight_country_China,num_feat_cat_feat_flight_country_DRAGON,num_feat_cat_feat_flight_country_Hong Kong SAR of China,num_feat_cat_feat_flight_country_Japan,num_feat_cat_feat_flight_country_Others,num_feat_cat_feat_flight_country_United States,num_feat_cat_feat_type_large_airport,num_feat_cat_feat_type_medium_airport,num_feat_cat_feat_type_small_airport,num_feat_cat_feat_iso_country_AU,num_feat_cat_feat_iso_country_CN,num_feat_cat_feat_iso_country_JP,num_feat_cat_feat_iso_country_Others,num_feat_cat_feat_iso_country_SG,num_feat_cat_feat_iso_country_TH,num_feat_cat_feat_iso_country_TW
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,UO,5536,RJBB,large_airport,Kansai International Airport,34.427299,135.244003,26.0,AS,JP,JP-27,Osaka,yes,RJBB,KIX,NaN,http://www.kansai-airport.or.jp/en/index.asp,http://en.wikipedia.org/wiki/Kansai_Internatio...,OSA,2774.0,Hong Kong Express Airways,\N,UO,HKE,HONGKONG SHUTTLE,Hong Kong SAR of China,Y,0,27,10,7,1,3,Friday,Hong Kong SAR of China,large_airport,3,1,34.427299,135.244003,JP,27,10,7,1,3,3,1,34.427299,135.244003,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,CI,5522,RCNN,medium_airport,Tainan Airport,22.950399,120.206001,63.0,AS,TW,TW-TNN,Tainan City,yes,RCNN,TNN,NaN,http://www.tna.gov.tw/,http://en.wikipedia.org/wiki/Tainan_Airport,"臺南機場, 臺南航空站",1756.0,China Airlines,\N,CI,CAL,DYNASTY,Taiwan,Y,0,17,11,4,23,4,Thursday,Others,medium_airport,2,0,22.950399,120.206001,TW,17,11,4,23,4,2,0,22.950399,120.206001,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1


In [39]:
num_feature= [y for y in df.columns if y[0:9]=='num_feat_']

In [40]:
test=df['Target']
# train = df[.drop('Target',axis= 1)]
train = df[feature]

In [41]:
df.Target.value_counts(normalize=True)

0    0.956165
1    0.043835
Name: Target, dtype: float64

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df[num_feature], df['Target'] ,test_size = 0.333, random_state = 0)

In [43]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid,valid_y):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict_proba(feature_vector_valid)[:,1]
    
    predictions_amount =predictions * 800
    
    mean_squared_error = sum((predictions_amount-y_test*800)**2)/len(predictions_amount)
    
    mean_absolute_error = sum(abs(predictions_amount-y_test*800))/len(predictions_amount)
    
    return predictions,mean_squared_error,mean_absolute_error
#     return roc_auc_score(predictions, valid_y), predictions_amount

In [44]:
# Linear Classifier
prob,Q2,Q1 = train_model(LogisticRegression(), X_train, y_train, X_test, y_test)
print "LR mean_squared_error: ", Q2, "LR mean_absolute_error: ", Q1

LR mean_squared_error:  25811.3182593 LR mean_absolute_error:  65.3232437094


In [45]:
# SVM
# prob,Q2,Q1 = train_model(SVC(), X_train, y_train, X_test, y_test)
# print "SVM mean_squared_error: ", Q2, "SVM mean_absolute_error: ", Q1

In [46]:
# DTree
prob,Q2,Q1 = train_model(tree.DecisionTreeClassifier(),X_train, y_train, X_test, y_test)
print "RF mean_squared_error: ", Q2, "RF mean_absolute_error: ", Q1

RF mean_squared_error:  33322.2265642 RF mean_absolute_error:  42.8489838179


In [47]:
# RF 
prob,Q2,Q1 = train_model(RandomForestClassifier(),X_train, y_train, X_test, y_test)
print "RF mean_squared_error: ", Q2, "RF mean_absolute_error: ", Q1

RF mean_squared_error:  24487.7317061 RF mean_absolute_error:  51.1497059355


In [48]:
#XGboost
prob,Q2,Q1 = train_model(xgboost.XGBClassifier(),X_train, y_train, X_test, y_test)
print "Xgb mean_squared_error: ", Q2, "Xgb mean_absolute_error: ", Q1

Xgb mean_squared_error:  25039.7746823 Xgb mean_absolute_error:  63.9624071176


In [ ]:
#LightGBM

In [49]:
for c in df[cat_feature]:
    col_type = df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df[c] = df[c].astype('category')

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df[feature+cat_feature], df['Target'] ,test_size = 0.333, random_state = 0)

In [57]:
feature = feature+cat_feature

In [64]:
lgb_train = lgb.Dataset(X_train, label = y_train, categorical_feature = cat_feature,free_raw_data=False )
lgb_eval = lgb.Dataset(X_test, label = y_test, categorical_feature = cat_feature, reference=lgb_train)

# params = { 
#     'objective': 'binary', 
#     'metric': 'binary_logloss', 
#     'verbose': -1 
# } 

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params, lgb_train ,num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5) 
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print   'mean_squared_error = ', sum((y_pred-y_test*800)**2)/len(y_pred)
print   'mean_absolute_error = ', sum(abs(y_pred-y_test*800))/len(y_pred)

/usr/local/lib/python2.7/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python2.7/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's l2: 0.229518	valid_0's auc: 0.712999
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.211062	valid_0's auc: 0.71484
[3]	valid_0's l2: 0.194404	valid_0's auc: 0.715835
[4]	valid_0's l2: 0.1794	valid_0's auc: 0.71718
[5]	valid_0's l2: 0.165841	valid_0's auc: 0.727288
[6]	valid_0's l2: 0.153637	valid_0's auc: 0.729109
[7]	valid_0's l2: 0.142593	valid_0's auc: 0.730013
[8]	valid_0's l2: 0.132631	valid_0's auc: 0.730155
[9]	valid_0's l2: 0.123631	valid_0's auc: 0.730736
[10]	valid_0's l2: 0.115498	valid_0's auc: 0.730241
[11]	valid_0's l2: 0.108162	valid_0's auc: 0.732273
[12]	valid_0's l2: 0.101529	valid_0's auc: 0.733043
[13]	valid_0's l2: 0.0955316	valid_0's auc: 0.732907
[14]	valid_0's l2: 0.0901062	valid_0's auc: 0.732763
[15]	valid_0's l2: 0.0852054	valid_0's auc: 0.73244
[16]	valid_0's l2: 0.0807797	valid_0's auc: 0.734328
[17]	valid_0's l2: 0.0767702	valid_0's auc: 0.734974
[18]	valid_0's l2: 0.0731543	valid_0's auc: 0.739257
[19]	valid_

In [65]:
gbm = lgb.train(params, lgb_train ,num_boost_round=2000,
                valid_sets=lgb_eval,
            early_stopping_rounds=5) 
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print   'mean_squared_error = ', sum((y_pred-y_test*800)**2)/len(y_pred)
print   'mean_absolute_error = ', sum(abs(y_pred-y_test*800))/len(y_pred)

/usr/local/lib/python2.7/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	valid_0's l2: 0.229518	valid_0's auc: 0.712999
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.211062	valid_0's auc: 0.71484
[3]	valid_0's l2: 0.194404	valid_0's auc: 0.715835
[4]	valid_0's l2: 0.1794	valid_0's auc: 0.71718
[5]	valid_0's l2: 0.165841	valid_0's auc: 0.727288
[6]	valid_0's l2: 0.153637	valid_0's auc: 0.729109
[7]	valid_0's l2: 0.142593	valid_0's auc: 0.730013
[8]	valid_0's l2: 0.132631	valid_0's auc: 0.730155
[9]	valid_0's l2: 0.123631	valid_0's auc: 0.730736
[10]	valid_0's l2: 0.115498	valid_0's auc: 0.730241
[11]	valid_0's l2: 0.108162	valid_0's auc: 0.732273
[12]	valid_0's l2: 0.101529	valid_0's auc: 0.733043
[13]	valid_0's l2: 0.0955316	valid_0's auc: 0.732907
[14]	valid_0's l2: 0.0901062	valid_0's auc: 0.732763
[15]	valid_0's l2: 0.0852054	valid_0's auc: 0.73244
[16]	valid_0's l2: 0.0807797	valid_0's auc: 0.734328
[17]	valid_0's l2: 0.0767702	valid_0's auc: 0.734974
[18]	valid_0's l2: 0.0731543	valid_0's auc: 0.739257
[19]	valid_

In [66]:
gbm = lgb.train(params, lgb_train ,num_boost_round=2000,
                valid_sets=lgb_eval,
            early_stopping_rounds=10) 
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print   'mean_squared_error = ', sum((y_pred-y_test*800)**2)/len(y_pred)
print   'mean_absolute_error = ', sum(abs(y_pred-y_test*800))/len(y_pred)

/usr/local/lib/python2.7/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[1]	valid_0's l2: 0.229518	valid_0's auc: 0.712999
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.211062	valid_0's auc: 0.71484
[3]	valid_0's l2: 0.194404	valid_0's auc: 0.715835
[4]	valid_0's l2: 0.1794	valid_0's auc: 0.71718
[5]	valid_0's l2: 0.165841	valid_0's auc: 0.727288
[6]	valid_0's l2: 0.153637	valid_0's auc: 0.729109
[7]	valid_0's l2: 0.142593	valid_0's auc: 0.730013
[8]	valid_0's l2: 0.132631	valid_0's auc: 0.730155
[9]	valid_0's l2: 0.123631	valid_0's auc: 0.730736
[10]	valid_0's l2: 0.115498	valid_0's auc: 0.730241
[11]	valid_0's l2: 0.108162	valid_0's auc: 0.732273
[12]	valid_0's l2: 0.101529	valid_0's auc: 0.733043
[13]	valid_0's l2: 0.0955316	valid_0's auc: 0.732907
[14]	valid_0's l2: 0.0901062	valid_0's auc: 0.732763
[15]	valid_0's l2: 0.0852054	valid_0's auc: 0.73244
[16]	valid_0's l2: 0.0807797	valid_0's auc: 0.734328
[17]	valid_0's l2: 0.0767702	valid_0's auc: 0.734974
[18]	valid_0's l2: 0.0731543	valid_0's auc: 0.739257
[19]	valid

In [ ]:
#Lightgbl best performance based on both Q1 and Q2 error, both lowest

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
gridParams = {
    'task': ['train'],
    'boosting_type': ['gbdt'],
    'objective': ['binary'],
    'metric': [{'l2', 'auc'}],
    'learning_rate': [0.005],
    'n_estimators': [40],
    'num_leaves': [6,8,12,16],
    'random_state' : [501], # Updated from 'seed'
    'colsample_bytree' : [0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4]
    }

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid = GridSearchCV(lgb.LGBMClassifier(), gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=2)

In [ ]:
# Run the grid
grid.fit(X_train,y_train)
# grid,fit(X_train,y_train, feature_name = feature+cat_feature, categorical_feature = cat_feature)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
grid,fit(X_train,y_train, feature_name = feature+cat_feature, categorical_feature = cat_feature)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)